<a href="https://colab.research.google.com/github/GerardoG2/CS4210-Assignment2/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning

### Set Up

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Change Working Directory

In [16]:
%cd /content/drive/MyDrive/CEOBusinessChallenge/Notebooks


/content/drive/MyDrive/CEOBusinessChallenge/Notebooks


In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
pd.set_option('display.max_columns', None)

## Create General Functions

Function for sorting data by Fiscal Year and month

In [19]:
def sort_fiscal_year(df):
    """
    Sorts a DataFrame by Fiscal Year and Month (July–June fiscal order).
    Assumes columns 'Fiscal Year' and 'Month' exist.
    Returns a new sorted DataFrame.
    """
    # Define fiscal year month order (July - June)
    month_order = [
        'July', 'August', 'September', 'October', 'November', 'December',
        'January', 'February', 'March', 'April', 'May', 'June'
    ]

    # Convert Month column to ordered categorical type
    df = df.copy()  # avoid modifying the original DataFrame
    df['Month'] = pd.Categorical(df['Month'], categories=month_order, ordered=True)

    # Sort and reset index
    df = df.sort_values(['Fiscal Year', 'Month']).reset_index(drop=True)

    return df

In [75]:
def get_calendar_year_from_fiscal_year(df):
  """
  Get year from 'Fiscal year'and 'Month' column.
  Returns a series.
  """
  month = df['Month'] = df['Month'].astype(str).str.strip().str.capitalize()

  mask = month.isin(['July', 'August', 'September', 'October', 'November', 'December'])

  return df['Fiscal Year'] - mask.astype(int)

In [148]:
def get_fiscal_year_from_calendar_year(df):
  """
  Get fiscal year from 'Calendar year'and 'Month' column.
  Takes a dataframe as input and returns a series.
  """
  month = df['Month'] = df['Month'].astype(str).str.strip().str.capitalize()

  mask = month.isin(['July', 'August', 'September', 'October', 'November', 'December'])

  return df['Calendar Year'] + mask.astype(int)

In [100]:
def sort_by_calendar_year_month(df):
  """
  Sort dataframe by calendar year and month
  """
  month_order = [
        'January', 'February', 'March', 'April', 'May', 'June','July',
        'August', 'September', 'October', 'November', 'December',
    ]
  df['Month'] = pd.Categorical(df['Month'], categories=month_order, ordered=True)
  df = df.sort_values(by=['Calendar Year', 'Month']).reset_index(drop=True)
  return df

# Load Data

## Parking Revenue

Clean & Reformat Data

In [101]:

def load_parking_revenue_data(file_path):
  """
  Load and Merge Parking Revenue Files
  """
  parking_revenue_df = pd.DataFrame()

  for year in range(2018, 2026):
        df_pr = pd.read_excel(file_path,
                      sheet_name = f'{year}',
                      skiprows=[0])

        df_pr =df_pr.dropna(how='all') # drop empty rows

        df_pr = df_pr.rename(columns={year:'Parking'}) # rename column
        df_pr['Fiscal Year'] = pd.Series(data=[year]*len(df_pr)).astype('Int64') # add column for year

        df_pr = df_pr[ (df_pr['Parking'] != 'Total') & (df_pr['Parking'] != 'Total Revenue')] # drop aggregated rows


        # convert to long format to facilitate analysis
        df_pr = pd.melt(df_pr,
                        id_vars=['Fiscal Year','Parking'],
                        value_vars=['July', 'August',
                        'September', 'October', 'November','December',
                        'January', 'February', 'March', 'April',
                        'May', 'June'],
                        var_name = 'Month',
                        value_name = 'Revenue'

        )

        df_pr['Revenue'] = df_pr['Revenue'].fillna(0)

        df_pr['Revenue'] = df_pr['Revenue'].round().astype('Int64')

        df_pr['Calendar Year'] = get_calendar_year_from_fiscal_year(df_pr)
        df_pr = sort_by_calendar_year_month(df_pr)

        # Add Calendar Year
        # Reorder columns
        new_column_order = ['Fiscal Year', 'Calendar Year', 'Month', 'Parking','Revenue']
        df_pr = df_pr[new_column_order]

        parking_revenue_df = pd.concat([parking_revenue_df, df_pr]).reset_index(drop=True)


        # # sort by fiscal year and month
        # parking_revenue_df = sort_fiscal_year(parking_revenue_df)


  return parking_revenue_df

Load Data

In [102]:
file_path = '../Data/RawData/ONT Parking Revenue 2018-2025.xlsx'

df_pr= load_parking_revenue_data(file_path)
df_pr

,Fiscal Year,Calendar Year,Month,Parking,Revenue
0,2018,2017,July,LOT 2,257917
1,2018,2017,July,LOT 3,154585
2,2018,2017,July,LOT 4,693701
3,2018,2017,July,LOT 5,470626
4,2018,2017,July,VALET,21675
...,...,...,...,...,...
595,2025,2025,June,LOT 4,1382761
596,2025,2025,June,LOT 5,248942
597,2025,2025,June,LOT 6,194330
598,2025,2025,June,VALET,11160


Check Missing Values Before Downloading

In [103]:
df_pr.isna().sum()

,0
Fiscal Year,0
Calendar Year,0
Month,0
Parking,0
Revenue,0


download clean dataframe

In [104]:
df_pr.to_csv('../Data/CleanData/ParkingRevenue2018-2025.csv', index = False)

## Parking Transactions

In [105]:
def load_parking_transaction_data(file_path):
  """
  Load and Merge Parking Transaction Files
  """
  parking_transaction_df = pd.DataFrame()

  for year in range(2018, 2026):
        df = pd.read_excel(file_path,
                      sheet_name = f'{year}',
                      skiprows=[0])

        df =df.dropna(how='all') # drop empty rows

        df = df.rename(columns={year:'Parking'}) # rename column
        df['Fiscal Year'] = pd.Series(data=[year]*len(df_pr)).astype('Int64') # add column for year

        df = df[ (df['Parking'] != 'Total') & (df['Parking'] != 'Total Transactions')] # drop aggregated rows


        # convert to long format to facilitate analysis
        df = pd.melt(df,
                        id_vars=['Fiscal Year','Parking'],
                        value_vars=['July', 'August',
                        'September', 'October', 'November','December',
                        'January', 'February', 'March', 'April',
                        'May', 'June'],
                        var_name = 'Month',
                        value_name = 'Transactions'

        )
        df['Transactions'] = df['Transactions'].fillna(0)
        df['Transactions'] = df['Transactions'].round().astype('Int64')

        # Add Calendar Year Column
        df['Calendar Year'] = get_calendar_year_from_fiscal_year(df)
        df = sort_by_calendar_year_month(df)

        # Reorder columns
        new_column_order = ['Fiscal Year', 'Calendar Year', 'Month', 'Parking','Transactions']
        df = df[new_column_order]

        parking_transaction_df = pd.concat([parking_transaction_df, df]).reset_index(drop=True)

        # sort by fiscal year and month
        parking_transaction_df = sort_fiscal_year(parking_transaction_df)
  return parking_transaction_df

In [106]:
file_path = '../Data/RawData/ONT Parking Transactions 2018-2025 .xlsx'

df_transactions = load_parking_transaction_data(file_path)
df_transactions

,Fiscal Year,Calendar Year,Month,Parking,Transactions
0,2018,2017,July,LOT 2,0
1,2018,2017,July,LOT 3,0
2,2018,2017,July,LOT 4,0
3,2018,2017,July,LOT 5,0
4,2018,2017,July,VALET,0
...,...,...,...,...,...
595,2025,2025,June,LOT 4,21830
596,2025,2025,June,LOT 5,3242
597,2025,2025,June,LOT 6,2581
598,2025,2025,June,VALET,105


Check missing values before downloading

In [107]:
df_transactions.isna().sum()

,0
Fiscal Year,0
Calendar Year,0
Month,0
Parking,0
Transactions,0


Download clean dataframe

In [108]:
df_transactions.to_csv('../Data/CleanData/ParkingTransactions2018-2025.csv', index = False)

## Passenger Statistics

Load data for monthly passengers per airline from 2018 - 2025.

In [109]:
file_path = '../Data/RawData/ONT Passenger Statistics 2018-2025.xlsx'

df_passengers_per_airline = pd.read_excel(file_path, sheet_name = 'Passengers')
df_passengers_per_airline.rename(columns={'Year':'Fiscal Year'}, inplace=True)
df_passengers_per_airline = sort_fiscal_year(df_passengers_per_airline)

# Add Calendar Year Column
df_passengers_per_airline['Calendar Year'] = get_calendar_year_from_fiscal_year(df_passengers_per_airline)
df_passengers_per_airline = sort_by_calendar_year_month(df_passengers_per_airline)

# sort by calendar year and month
new_column_order = ['Fiscal Year', 'Calendar Year', 'Month', 'Airline','Passengers']
df_passengers_per_airline = df_passengers_per_airline[new_column_order]

df_passengers_per_airline

,Fiscal Year,Calendar Year,Month,Airline,Passengers
0,2018,2017,July,Aeromexico,0
1,2018,2017,July,Alaska Airlines,36577
2,2018,2017,July,American Airlines,85132
3,2018,2017,July,China Airlines,14917
4,2018,2017,July,Delta Air Lines,16251
...,...,...,...,...,...
1117,2025,2025,June,Starlux Airlines,7743
1118,2025,2025,June,United Airlines,68229
1119,2025,2025,June,Volaris,17684
1120,2025,2025,June,Volaris El Salvador,0


Check missing values

In [91]:
df_passengers_per_airline.isna().sum()

,0
Fiscal Year,0
Calendar Year,0
Month,0
Airline,0
Passengers,0


Load data for monthly inbound/outbound passengers at ONT from 2018-2025

In [110]:
file_path = '../Data/RawData/ONT Passenger Statistics 2018-2025.xlsx'

df_inbound_outbound = pd.read_excel(file_path, sheet_name = 'Inbound Outbound')

In [111]:
# rename columns
df_inbound_outbound = df_inbound_outbound.rename(columns={'Year':'Fiscal Year','Inbound (deplaned)':'Inbound', 'Outbound (enplaned)':'Outbound'})

In [112]:
# add total passengers column
df_inbound_outbound['Total Passengers'] = (
    df_inbound_outbound['Inbound'] + df_inbound_outbound['Outbound']
)


In [113]:
# Add Calendar Year Column
df_inbound_outbound['Calendar Year'] = get_calendar_year_from_fiscal_year(df_inbound_outbound)
df_inbound_outbound = sort_by_calendar_year_month(df_inbound_outbound)

# Reorder columns
new_column_order = ['Fiscal Year', 'Calendar Year', 'Month', 'Inbound', 'Outbound', 'Total Passengers']
df_inbound_outbound = df_inbound_outbound[new_column_order]

df_inbound_outbound

,Fiscal Year,Calendar Year,Month,Inbound,Outbound,Total Passengers
0,2018,2017,July,227192,221107,448299
1,2018,2017,August,231693,226336,458029
2,2018,2017,September,206007,207837,413844
3,2018,2017,October,228481,226791,455272
4,2018,2017,November,227176,228846,456022
...,...,...,...,...,...,...
87,2025,2025,February,223937,226156,450093
88,2025,2025,March,280024,279595,559619
89,2025,2025,April,290950,284673,575623
90,2025,2025,May,318435,311702,630137


In [114]:
# check missing values
df_inbound_outbound.isna().sum()

,0
Fiscal Year,0
Calendar Year,0
Month,0
Inbound,0
Outbound,0
Total Passengers,0


download individual csv for each passenger statistic dataframe

In [115]:
df_passengers_per_airline.to_csv('../Data/CleanData/PassengerStatisticsPerAirline2018-2025.csv', index = False)

In [116]:
df_inbound_outbound.to_csv('../Data/CleanData/InboundOutboundPassengers2018-2025.csv', index = False)

Load data and convert to csv: Passenger Totals from 2007 - 2024

In [123]:
# this data uses fiscal year
df_yearly_passengers = pd.read_excel('../Data/RawData/ONT Passenger Totals 2007-2024.xlsx')
df_yearly_passengers.rename(columns={'Year':'Fiscal Year'}, inplace=True)
df_yearly_passengers = df_yearly_passengers.sort_values(by=['Fiscal Year']).reset_index(drop=True)

df_yearly_passengers

,Fiscal Year,Passengers Total
0,2007,7207150
1,2008,6232975
2,2009,4861110
3,2010,4812578
4,2011,4540694
5,2012,4296459
6,2013,3971136
7,2014,4127280
8,2015,4209311
9,2016,4251903


In [124]:
df_yearly_passengers.to_csv('../Data/CleanData/PassengerTotals2007-2024.csv', index = False)

## Transportation Network Company Data

In [131]:
def load_tnc_trip_count_data(file_path):
  """
  Load and Merge Transportation Network Company Files
  """
  tnc_trip_count_df = pd.DataFrame()

  for year in range(2021, 2024):
    tnc_df = pd.read_excel(file_path, usecols='A:M', sheet_name = f'{year}')
    tnc_df.columns = range(tnc_df.shape[1])
    tnc_df = tnc_df.dropna(how='all')
    tnc_df = tnc_df.fillna(0).reset_index(drop=True)

    months = ['July', 'August', 'September', 'October', 'November', 'December',
              'January', 'February', 'March', 'April', 'May', 'June']
    rename_dict = {i: month for i, month in enumerate(months, start=1)}
    tnc_df = tnc_df.rename(columns=rename_dict)

    tnc_df['TNC'] = tnc_df[0].where(~tnc_df[0].isin(['Pickups', 'Dropoffs', 'Totals', 'Total Fees']))
    tnc_df['TNC'] = tnc_df['TNC'].ffill()

    tnc_df = tnc_df [ ~tnc_df[0].isin(tnc_df['TNC']) & ~tnc_df[0].isin(['Totals', 'Total Fees']) ]

    tnc_df.rename(columns={0:'Trip Type'}, inplace=True)

    tnc_df = pd.melt(tnc_df,
                    id_vars = ['TNC', 'Trip Type'],
                    value_vars = months,
                    value_name = 'Trip Count',
                    var_name='Month')

    tnc_df['Fiscal Year'] = year

    tnc_df = tnc_df[['Fiscal Year', 'Month', 'TNC', 'Trip Type', 'Trip Count']]

   # add calendar year
    tnc_df['Calendar Year'] = get_calendar_year_from_fiscal_year(tnc_df)
    tnc_df = sort_by_calendar_year_month(tnc_df)

    # rearrange columns
    new_column_order = ['Fiscal Year', 'Calendar Year', 'Month', 'TNC', 'Trip Type', 'Trip Count']
    tnc_df = tnc_df[new_column_order]

    tnc_trip_count_df = pd.concat([tnc_trip_count_df, tnc_df]).reset_index(drop=True)

  return tnc_trip_count_df

In [132]:
file_path = '../Data/RawData/ONT TNC Data 2020-2023.xlsx'
tnc_trips_df = load_tnc_trip_count_data(file_path)

tnc_trips_df


,Fiscal Year,Calendar Year,Month,TNC,Trip Type,Trip Count
0,2021,2020,July,Lyft,Pickups,4664
1,2021,2020,July,Lyft,Dropoffs,4392
2,2021,2020,July,Uber,Pickups,0
3,2021,2020,July,Uber,Dropoffs,0
4,2021,2020,July,Wingz,Pickups,0
...,...,...,...,...,...,...
283,2023,2023,June,Uber,Dropoffs,0
284,2023,2023,June,Wingz,Pickups,0
285,2023,2023,June,Wingz,Dropoffs,0
286,2023,2023,June,Opoli,Pickups,0


In [133]:
# final check for missing data
tnc_trips_df.isna().sum()


,0
Fiscal Year,0
Calendar Year,0
Month,0
TNC,0
Trip Type,0
Trip Count,0


Create new dataframe with total fees from TNCs

In [134]:
tnc_fees_df = tnc_trips_df.groupby(['Fiscal Year', 'Month', 'TNC'])['Trip Count'].sum().reset_index()
tnc_fees_df.rename(columns = {'Trip Count':'Total Trips'}, inplace=True)
tnc_fees_df['Total Fees'] = tnc_fees_df['Total Trips']*4

# add calendar year
tnc_fees_df['Calendar Year'] = get_calendar_year_from_fiscal_year(tnc_fees_df)
tnc_fees_df = sort_by_calendar_year_month(tnc_fees_df)

# rearrange columns
tnc_fees_df = tnc_fees_df[['Fiscal Year', 'Calendar Year', 'Month', 'TNC', 'Total Trips', 'Total Fees']]

tnc_fees_df

,Fiscal Year,Calendar Year,Month,TNC,Total Trips,Total Fees
0,2021,2020,July,Lyft,9056,36224
1,2021,2020,July,Opoli,0,0
2,2021,2020,July,Uber,0,0
3,2021,2020,July,Wingz,0,0
4,2021,2020,August,Lyft,9742,38968
...,...,...,...,...,...,...
139,2023,2023,May,Wingz,16,64
140,2023,2023,June,Lyft,0,0
141,2023,2023,June,Opoli,0,0
142,2023,2023,June,Uber,0,0


Donwload a csv for both clean dataframes


*   Trip Type Count Per TNC
*   Total Trips and Fees per TNC



In [135]:
tnc_trips_df.to_csv('../Data/CleanData/TNC_TripType_Counts2021-2023.csv', index = False)
tnc_fees_df.to_csv('../Data/CleanData/TNC_TotalTripsAndFees2021-2023.csv', index = False)

## Parking Lot Data

### 2020 Daily Lot Counts

Function to read all daily lot counts and merge into one file

In [44]:
def load_lot_count_data(file_path):
  """
  Load and Merge Parking Lot Count Data
  """


In [149]:
def get_year_month(file_path):
  """
  Get year and month from excel file path as a tuple.
  """
  file_path_lower = file_path.lower() # Convert the string to lowercase

  if 'january' in file_path_lower:
    month = 'January'
  elif 'february' in file_path_lower:
    month = 'February'
  elif 'march' in file_path_lower:
    month = 'March'
  elif 'april' in file_path_lower:
    month = 'April'
  elif 'may' in file_path_lower:
    month = 'May'
  elif 'june' in file_path_lower:
    month = 'June'
  elif 'july' in file_path_lower:
    month = 'July'
  elif 'august' in file_path_lower:
    month = 'August'
  elif 'september' in file_path_lower:
    month = 'September'
  elif 'october' in file_path_lower:
    month = 'October'
  elif 'november' in file_path_lower:
    month = 'November'
  elif 'december' in file_path_lower:
    month = 'December'
  else:
      month = None


  if '2020' in file_path:
    year = 2020
  elif '2021' in file_path:
    year = 2021
  elif '2022' in file_path:
    year = 2022
  elif '2023' in file_path:
    year = 2023
  else:
      year = None


  return year, month

In [45]:
file_path = '../Data/RawData/ONT Lot Counts 2020-2023/2020/PCI ONT Daily Lot Counts October 2020.xlsx'
lot_count_df = pd.read_excel(file_path, header = None)
lot_count_df.dropna(how='all', inplace=True) # drop empty rows
lot_count_df[1] = lot_count_df[1].ffill() # fill NaT cells with dates
lot_count_df = lot_count_df[ ~lot_count_df[0].isna()]


lot_count_df = lot_count_df[ ~(lot_count_df[2]).astype(str).str.lower().str.strip().str.contains('number of spaces')]

new_column_names = ['Lot', 'Date', 'Spaces', 'Vacancies', 'Occupied', '% Capacity Filled']
new_column_names = dict(zip((i for i in range(6)), new_column_names))
lot_count_df.rename(columns=new_column_names, inplace=True)

lot_count_df['Occupied'].fillna(0,inplace=True)
lot_count_df['Day of the Week'] = lot_count_df['Date'].dt.day_name()
lot_count_df['Month'] = lot_count_df['Date'].dt.month_name()

new_column_order = ['Date', 'Day of the Week', 'Month',
                    'Lot', 'Spaces', 'Vacancies',
                    'Occupied', '% Capacity Filled',6, 7]
lot_count_df = lot_count_df[new_column_order]

lot_count_df['% Capacity Filled'] = lot_count_df['% Capacity Filled'].apply(lambda x: round(x*100, 2))
lot_count_df

# move total revenue amounts to new dataframe
daily_lot_revenue_df = lot_count_df.where(lot_count_df[6].astype(str).str.lower().str.contains('revenue')).reset_index(drop=True)
daily_lot_revenue_df = daily_lot_revenue_df.dropna(how='all').reset_index(drop=True)
daily_lot_revenue_df = daily_lot_revenue_df[['Date', 'Day of the Week', 'Month', 6, 7]]
daily_lot_revenue_df = daily_lot_revenue_df.pivot(index=['Date', 'Day of the Week', 'Month'],
                                                  columns=6,
                                                  values = 7).reset_index()
daily_lot_revenue_df = daily_lot_revenue_df.rename(
    columns={
        'Total C/C Revenue': 'Credit Card Revenue',
        'Total Cash Revenue': 'Cash Revenue'
    }
)

daily_lot_revenue_df['Cash Revenue'] = daily_lot_revenue_df['Cash Revenue'].astype('Int64')
daily_lot_revenue_df['Credit Card Revenue'] = daily_lot_revenue_df['Credit Card Revenue'].astype('Int64')
daily_lot_revenue_df['Total Revenue'] = daily_lot_revenue_df['Cash Revenue'] + daily_lot_revenue_df['Credit Card Revenue']

daily_lot_revenue_df

lot_count_df.drop(columns=[6,7], inplace=True)

lot_count_df.head(15)

/tmp/ipython-input-256576412.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  lot_count_df['Occupied'].fillna(0,inplace=True)
/tmp/ipython-input-256576412.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lot_count_df['Occupied'].fillna(0,inplace=True)


,Date,Day of the Week,Month,Lot,Spaces,Vacancies,Occupied,% Capacity Filled
1,2020-10-01,Thursday,October,LOT 2 General,1219,927,292,23.95
2,2020-10-01,Thursday,October,LOT 2 Premium,324,286,38,11.73
3,2020-10-01,Thursday,October,LOT 3,1192,763,429,35.99
4,2020-10-01,Thursday,October,LOT 4 General,1430,862,568,39.72
5,2020-10-01,Thursday,October,LOT 4 Premium,352,174,178,50.57
6,2020-10-01,Thursday,October,LOT 5,2200,2200,0,0.00
11,2020-10-02,Friday,October,LOT 2 General,1219,889,330,27.07
12,2020-10-02,Friday,October,LOT 2 Premium,324,274,50,15.43
13,2020-10-02,Friday,October,LOT 3,1192,702,490,41.11
14,2020-10-02,Friday,October,LOT 4 General,1430,835,595,41.61


In [46]:
daily_lot_revenue_df

6,Date,Day of the Week,Month,Credit Card Revenue,Cash Revenue,Total Revenue
0,2020-10-01,Thursday,October,27179,2434,29613
1,2020-10-02,Friday,October,38705,2502,41207
2,2020-10-03,Saturday,October,20749,3039,23788
3,2020-10-04,Sunday,October,38286,5824,44110
4,2020-10-05,Monday,October,35015,4506,39521
5,2020-10-06,Tuesday,October,22578,1936,24514
6,2020-10-07,Wednesday,October,22501,1450,23951


In [168]:
import pandas as pd

def load_lot_occupancy_and_revenue_data(file_path):
  """
  Load and Merge Parking Lot Count Data for an excel file containing a single months data.
  Returns two data frames: Lot Occupancy and Revenue.
  """
  year, month = get_year_month(file_path)
  print(year, month) # checking

  daily_lot_occupancy_df = pd.DataFrame() # initalized lot occupancy df
  daily_parking_revenue_df = pd.DataFrame() # initialize daily parking revenue df

  # read all sheets in the spreadsheet
  lot_count_data_dictionary = pd.read_excel(file_path, header = None, sheet_name=None)

  # iterate through all sheets in the excel sheet except the last (Summary)
  for sheet_name, lot_count_df in list(lot_count_data_dictionary.items())[:-1]:
    print(sheet_name) #checking
    lot_count_df.dropna(how='all', inplace=True) # drop empty rows
    # Make sure the date column is datetime
    lot_count_df[1] = pd.to_datetime(lot_count_df[1], errors='coerce')

    # Drop Excel epoch garbage
    lot_count_df.loc[lot_count_df[1] < pd.Timestamp("1990-01-01"), 1] = pd.NaT

    # Get the *first valid date* in the sheet (the earliest one usually tells the month)
    first_valid_date = lot_count_df[1].dropna().min()

    if pd.notna(first_valid_date):
        year_val = first_valid_date.year
        month_name = first_valid_date.strftime('%B')  # e.g. "October"

        # Decide the limit
        limit_val = 6 if (year_val < 2022 or (year_val == 2022 and month_name != 'November')) else 7
    else:
        # Default fallback if no valid date found
        limit_val = 6

    # Apply ffill with that limit
    lot_count_df[1] = lot_count_df[1].ffill(limit=limit_val)
    lot_count_df = lot_count_df.dropna(subset=[1])
    lot_count_df = lot_count_df[ ~lot_count_df[0].isna()]


    lot_count_df = lot_count_df[ ~(lot_count_df[2]).astype(str).str.lower().str.strip().str.contains('number of spaces')]

    new_column_names = ['Lot', 'Date', 'Spaces', 'Vacancies', 'Occupied', '% Capacity Filled']
    new_column_names = dict(zip((i for i in range(6)), new_column_names))
    lot_count_df.rename(columns=new_column_names, inplace=True)

    lot_count_df['Occupied'].fillna(0,inplace=True)
    lot_count_df['Day of the Week'] = lot_count_df['Date'].dt.day_name()
    lot_count_df['Month'] = lot_count_df['Date'].dt.month_name()

    # add fiscal year, calendar year columns
    lot_count_df['Calendar Year'] = year
    lot_count_df['Fiscal Year'] = get_fiscal_year_from_calendar_year(lot_count_df)


    new_column_order = ['Fiscal Year', 'Calendar Year', 'Date', 'Day of the Week', 'Month',
                        'Lot', 'Spaces', 'Vacancies',
                        'Occupied', '% Capacity Filled',6, 7]
    lot_count_df = lot_count_df[new_column_order]

    lot_count_df['% Capacity Filled'] = lot_count_df['% Capacity Filled'].apply(lambda x: round(x*100, 2))

    # drop rows for months and years not part of current spreadsheet
    lot_count_df = lot_count_df[lot_count_df['Month'] == month]
    lot_count_df = lot_count_df[lot_count_df['Calendar Year'] == year]

    # drop duplicates
    lot_count_df = lot_count_df.drop_duplicates()

    # drop missing dates
    lot_count_df = lot_count_df.dropna(subset=['Date'])

    # move total revenue amounts to new dataframe
    daily_lot_revenue_df = lot_count_df.where(lot_count_df[6].astype(str).str.lower().str.contains('revenue')).reset_index(drop=True)
    daily_lot_revenue_df = daily_lot_revenue_df.dropna(how='all').reset_index(drop=True)
    daily_lot_revenue_df = daily_lot_revenue_df[['Fiscal Year', 'Calendar Year',
                                                 'Date', 'Day of the Week', 'Month', 6, 7]]
    daily_lot_revenue_df = daily_lot_revenue_df.pivot(index=['Fiscal Year', 'Calendar Year',
                                                             'Date', 'Day of the Week', 'Month'],
                                                      columns=6,
                                                      values = 7).reset_index()
    daily_lot_revenue_df = daily_lot_revenue_df.rename(
        columns={
            'Total C/C Revenue': 'Credit Card Revenue',
            'Total Cash Revenue': 'Cash Revenue'
        }
    )

    daily_lot_revenue_df['Cash Revenue'] = daily_lot_revenue_df['Cash Revenue'].astype('Int64')
    daily_lot_revenue_df['Credit Card Revenue'] = daily_lot_revenue_df['Credit Card Revenue'].astype('Int64')
    daily_lot_revenue_df['Total Revenue'] = daily_lot_revenue_df['Cash Revenue'] + daily_lot_revenue_df['Credit Card Revenue']

    lot_count_df.drop(columns=[6,7], inplace=True)

    daily_lot_occupancy_df = pd.concat([daily_lot_occupancy_df, lot_count_df]).reset_index(drop=True)
    daily_parking_revenue_df = pd.concat([daily_parking_revenue_df, daily_lot_revenue_df]).reset_index(drop=True)

    # drop duplicate rows
    daily_lot_occupancy_df = daily_lot_occupancy_df.drop_duplicates()
    daily_parking_revenue_df = daily_parking_revenue_df.drop_duplicates()
  return daily_lot_occupancy_df, daily_parking_revenue_df


In [169]:
import warnings
warnings.filterwarnings("ignore")

file_path = '../Data/RawData/ONT Lot Counts 2020-2023/2020/PCI ONT Daily Lot Counts November 2020.xlsx'
daily_lot_occupancy_df, daily_parking_revenue_df = load_lot_occupancy_and_revenue_data(file_path)

2020 November
1-7
8-14
15-21
22-28
29 to end of the month


In [138]:
from pathlib import Path

def concat_all_yearly_lot_data(base_dir=Path('../Data/RawData/ONT Lot Counts 2020-2023')):
  """
  Iterate through each year directory and merge all monthly lot data into one dataframe.
  Returns two data frames in a tuple: Lot Occupancy and Revenue.
  """
  daily_lot_occupancy_df = pd.DataFrame() # initalized lot occupancy df
  daily_parking_revenue_df = pd.DataFrame() # initialize daily parking revenue df

  for file_path in base_dir.glob('**/*.xlsx'):
    if file_path.name.startswith('~$'):
      continue
    file_path = str(file_path)
    lot_count_df, lot_rev_df = load_lot_occupancy_and_revenue_data(file_path)
    daily_lot_occupancy_df = pd.concat([daily_lot_occupancy_df, lot_count_df]).reset_index(drop=True)
    daily_parking_revenue_df = pd.concat([daily_parking_revenue_df, lot_rev_df]).reset_index(drop=True)

    # sort dataframes
    daily_lot_occupancy_df = sort_by_calendar_year_month(daily_lot_occupancy_df)
    daily_parking_revenue_df = sort_by_calendar_year_month(daily_parking_revenue_df)

  return daily_lot_occupancy_df, daily_parking_revenue_df


In [170]:
import warnings
warnings.filterwarnings("ignore")

daily_lot_revenue_df, daily_parking_revenue_df = concat_all_yearly_lot_data()
daily_parking_revenue_df

2020 May
1-7


KeyError: '[7] not in index'

In [165]:
import warnings
warnings.filterwarnings("ignore")

file_path = '../Data/RawData/ONT Lot Counts 2020-2023/2020/PCI ONT Daily Lot Counts November 2020.xlsx'
daily_lot_occupancy_df, daily_parking_revenue_df = load_lot_occupancy_and_revenue_data(file_path)

2020 November
1-7
8-14
15-21
22-28
29 to end of the month


ValueError: Index contains duplicate entries, cannot reshape

In [145]:
daily_parking_revenue_df

6,Date,Day of the Week,Month,Credit Card Revenue,Cash Revenue,Total Revenue
0,2020-10-01,Thursday,October,27179,2434,29613
1,2020-10-02,Friday,October,38705,2502,41207
2,2020-10-03,Saturday,October,20749,3039,23788
3,2020-10-04,Sunday,October,38286,5824,44110
4,2020-10-05,Monday,October,35015,4506,39521
5,2020-10-06,Tuesday,October,22578,1936,24514
6,2020-10-07,Wednesday,October,22501,1450,23951
7,2020-10-08,Thursday,October,27304,1876,29180
8,2020-10-09,Friday,October,38763,2296,41059
9,2020-10-10,Saturday,October,3292,23285,26577


In [146]:
daily_lot_occupancy_df

,Date,Day of the Week,Month,Lot,Spaces,Vacancies,Occupied,% Capacity Filled
0,2020-10-01,Thursday,October,LOT 2 General,1219,927,292,23.95
1,2020-10-01,Thursday,October,LOT 2 Premium,324,286,38,11.73
2,2020-10-01,Thursday,October,LOT 3,1192,763,429,35.99
3,2020-10-01,Thursday,October,LOT 4 General,1430,862,568,39.72
4,2020-10-01,Thursday,October,LOT 4 Premium,352,174,178,50.57
...,...,...,...,...,...,...,...,...
181,2020-10-31,Saturday,October,LOT 2 Premium,324,277,47,14.51
182,2020-10-31,Saturday,October,LOT 3,1192,702,490,41.11
183,2020-10-31,Saturday,October,LOT 4 General,1430,902,528,36.92
184,2020-10-31,Saturday,October,LOT 4 Premium,352,184,168,47.73
